In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import load_model
import cv2
import numpy as np
from PIL import Image

# split the data among train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

# run 10 epochs of training.
batch_size = 128
num_classes = 10
epochs = 10

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")
model.save('mnist.h5')
print("Saving the model as mnist.h5")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(60000, 28, 28) (60000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 [==============================] - 130s 274ms/step - loss: 2.2891 - accuracy: 0.1405 - val_loss: 2.2263 - val_accuracy: 0.4132
Epoch 2/10
469/469 [==============================] - 111s 236ms/step - loss: 2.2159 - accuracy: 0.2914 - val_loss: 2.1251 - val_accuracy: 0.6517
Epoch 3/10
469/469 [==============================] - 129s 276ms/step - loss: 2.1144 - accuracy: 0.4498 - val_loss: 1.9811 - val_accuracy: 0.7228
Epoch 4/10
469/469 [==============================] - 143s 305ms/step - loss: 1.9722 - accuracy: 0.5406 - val_loss: 1.7790 - val_accuracy: 0.7471
Epoch 5/10
469/469 [==============================] - 172s 367ms/step - loss: 1.7783 - accuracy: 0.6015 - val_loss: 1.5247 - val_accuracy: 0.7662
Epoch 6/10
469/469 [==============================] - 176s 376ms/step - loss: 1.5468 - accuracy: 0.6390 - val_loss: 1.2559 - val_accuracy: 0.7876
Epoch 7/10
469/469 [======

In [11]:
model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels, the size of images in MNIST database
    img = img.resize((28,28))
    #convert to greyscale image
    img = img.convert('L')
    
    #display processed image
    #img.show()
    
    #convert the image to a numpy array
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

In [23]:
#analog photos

passes = 0

for n in range(10):
    fname = f"analog photos\image{n}.jpg"
    im = Image.open(fname)    
    digit, acc = predict_digit(im)
    
    if n == digit:
        passes += 1
    
    print(f"Actual: {n}\nKeras: {digit}\nCertainty: {acc*100}\n")
    
print (f"Passrate: {(passes/10)*100}%")


Actual: 0
Keras: 0
Certainty: 24.37974512577057

Actual: 1
Keras: 4
Certainty: 24.017217755317688

Actual: 2
Keras: 2
Certainty: 42.56452918052673

Actual: 3
Keras: 8
Certainty: 27.41352617740631

Actual: 4
Keras: 1
Certainty: 17.656411230564117

Actual: 5
Keras: 2
Certainty: 18.98147314786911

Actual: 6
Keras: 6
Certainty: 32.61137902736664

Actual: 7
Keras: 0
Certainty: 19.4354385137558

Actual: 8
Keras: 8
Certainty: 31.13182783126831

Actual: 9
Keras: 2
Certainty: 20.82875519990921

Passrate: 40.0%


In [27]:
#processed digit photos

passes = 0

for n in range(10):
    fname = "processed digit\{} correct.png".format(n)
    im = Image.open(fname)
    digit, acc = predict_digit(im)
    
    if n == digit:
        passes += 1

    print(f"Actual: {n}\nKeras: {digit}\nCertainty: {acc*100}\n")

print (f"Passrate: {(passes/10)*100}%")

Actual: 0
Keras: 0
Certainty: 36.80389225482941

Actual: 1
Keras: 4
Certainty: 51.46488547325134

Actual: 2
Keras: 2
Certainty: 81.21164441108704

Actual: 3
Keras: 8
Certainty: 39.96042013168335

Actual: 4
Keras: 0
Certainty: 20.153023302555084

Actual: 5
Keras: 5
Certainty: 24.620208144187927

Actual: 6
Keras: 6
Certainty: 39.3261194229126

Actual: 7
Keras: 2
Certainty: 28.99898886680603

Actual: 8
Keras: 8
Certainty: 52.32630372047424

Actual: 9
Keras: 8
Certainty: 45.522305369377136

Passrate: 50.0%


In [29]:
#paint photos

passes = 0

for n in range(10):
    fname = "paint photos\paint{}.png".format(n)
    im = Image.open(fname)
    digit, acc = predict_digit(im)
    
    if n == digit:
        passes += 1

    print(f"Actual: {n}\nKeras: {digit}\nCertainty: {acc*100}\n")
    
print (f"Passrate: {(passes/10)*100}%")

Actual: 0
Keras: 2
Certainty: 28.699281811714172

Actual: 1
Keras: 2
Certainty: 31.724023818969727

Actual: 2
Keras: 2
Certainty: 58.32288861274719

Actual: 3
Keras: 8
Certainty: 54.594409465789795

Actual: 4
Keras: 7
Certainty: 30.467376112937927

Actual: 5
Keras: 8
Certainty: 45.98771035671234

Actual: 6
Keras: 5
Certainty: 31.964462995529175

Actual: 7
Keras: 8
Certainty: 29.511478543281555

Actual: 8
Keras: 2
Certainty: 50.30859112739563

Actual: 9
Keras: 8
Certainty: 19.189681112766266

Passrate: 10.0%
